# IT Security - Sheet 2 "Symmetric Encryption"

**Total achievable points: 20**

**Released: 7.11.2026**

**Submission Deadline: 14.11.2025 13:00**

---
Groupnumber: 17

Names and matriculation numbers of **ALL** team members: 

Neo Ahrens (456647)

Christian Bick (456513)

Yorck Heilmann (456599)

---

**Important Information**

The assignments have to be submitted by groups of 4 students. Even if you are registered in RWTHmoodle to a submission group, **please include the group number as well as the name and matriculation number of every group member in this notebook**. In case you are not part of a submission group and want to hand in assignments, please contact `ba-itsec@itsec.rwth-aachen.de`.

Enter your solutions for the tasks in the respective cells of this notebook. These cells are either marked by "YOUR ANSWER HERE" or `#YOUR CODE HERE`. Do not add any new cells or remove existing ones, especially do not copy cells. Cells marked with `###PLAYGROUND` can be used to test your implementation and generate output (see example for the first tasks). Do not add any other output or tests in the cell of the task, just implement the function with the header provided. If you want to test your implementation, use the `###PLAYGROUND` cells. They will be ignored during grading. **Do not change any other cells or add new ones.**

All assertions provided by us should be considered part of the exercise description. Passing the assertions does not automatically mean that you'll get full points for the corresponding task, but failing the assertions pretty much guarantees that you'll get no points. 

**Do not import any further Python packages** except the default Python ones and the ones that are explicitly given by us.

## Content of this Assignment

In the lecture, you learned about symmetric cipher standards such as DES and AES. It was shown how to deal with the block ciphers in different modes of encryption. Also, you learned about hash functions, MDCs, and MACs. 
In this exercise, you'll take a deeper look into one of the modes presented in the lecture and be introduced to a new mode, the Output-Feedback-Mode. Also, you can again try to break some stuff (not the autograding of the assignments :D) by exploiting a bad MAC construction! But before, let's take a little detour to *One Time Pad* (OTP).

## 1. One-Time-Spy (6 Points)

You are working at a counter-intelligence agency, and your boss Eve currently has only one thing on her mind: The spies Alice and Bob. They cause havoc all the time, stealing sensitive information and sabotaging important facilities. And your agency has not caught them yet! Due to the immense pressure they are causing, Eve tries to intercept messages and mess around as much as possible. You are tasked with helping her.

Of course, every spy knows that the One-Time-Pad (OTP) is perfectly secure, and therefore only uses OTPs for communication. Knowing this, your predecessor has started implementing the OTP. Unfortunately, he suddenly died in an unexplained off-screen submarine accident (we choose to blame Johnny English). This leaves you with the following parts:

* **to_binary** takes a human readable string (such as "Hello World") and turns it into a "human readable" binary string, such as "010010000110010..."
* **from_binary** is its counterpart that turns a binary string back into a human-readable string.
* **xor_binary_messages** XORs two binary strings

In [1]:
def to_binary(message: str) -> str:
    binary_message = ''.join(format(ord(char), '08b') for char in message)
    return binary_message

def from_binary(binary_message: str) -> str:
    message = ''.join(chr(int(binary_message[i:i+8], 2)) for i in range(0, len(binary_message), 8))
    return message

def xor_binary_messages(first_message: str, second_message: str) -> str:
    if len(first_message) != len(second_message):
        raise ValueError("Binary strings must be of the same length")

    xor_result = ''.join('1' if bit1 != bit2 else '0' for bit1, bit2 in zip(first_message, second_message))
    return xor_result

assert xor_binary_messages('0011', '0101') == '0110'

### Task 1.1 Finishing the OTP (0.5 Points)

The first task given to you by Eve is to finish the implementation of the One-Time-Pad. Make sure it passes the test case below the Playground cell.

In [2]:
def encrypt(message: str, key: str) -> str:
    return xor_binary_messages(to_binary(message),key)

def decrypt(cipher: str, key: str) -> str:
    return from_binary(xor_binary_messages(cipher,key))
    

In [3]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.
print(encrypt("Hello World!", '010010001110011101100011101111110011010101100000101000010101111000001011101001011111100100111001'))
print(decrypt("000000001000001000001111110100110101101001000000111101100011000101111001110010011001110100011000", '010010001110011101100011101111110011010101100000101000010101111000001011101001011111100100111001'))

000000001000001000001111110100110101101001000000111101100011000101111001110010011001110100011000
Hello World!


In [4]:
plain = 'Hello World!'
key = '010010001110011101100011101111110011010101100000101000010101111000001011101001011111100100111001'
cipher = '000000001000001000001111110100110101101001000000111101100011000101111001110010011001110100011000'

encr = encrypt(plain, key)
assert encr == cipher
decr = decrypt(cipher, key)
assert decr == plain
# Your code is not necessarily correct just because it passes this test.

# Ensures that we do not have confusing "key" variables in the cells below.
del plain
del key
del cipher

In [5]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 1.2 - Confusing the spies (2 Points)

Alice and Bob want to meetup to transfer some very important documents. Hence, Alice encrypts her date using the OTP, sends the ciphertext to Bob, who then decrypts the date.

Ideally (for them), the transfer would look like this:

In [6]:
secret_key = '00000001101100110110101110010000101001011000000100101100100010000101111011011100'
true_date_plain = '24.12.2025'

encrypted_date = encrypt(true_date_plain, secret_key) # Alice encrypts the date.
print(f"Alice sends the encrypted message: {encrypted_date}")

received_message = encrypted_date # Ideally, Alice can send the message to Bob without Eve intercepting it.
print(f"Bob receives the encrypted message: {received_message}")

decrypted_date = decrypt(received_message, secret_key) # Bob decrypts the date.
print(f"Bob received the date \"{decrypted_date}\".")

# Our counter intelligence agency does not know the secret key, nor the true date.
del secret_key
del true_date_plain
del decrypted_date

Alice sends the encrypted message: 00110011100001110100010110100001100101111010111100011110101110000110110011101001
Bob receives the encrypted message: 00110011100001110100010110100001100101111010111100011110101110000110110011101001
Bob received the date "24.12.2025".


However, Eve was able to intercept the message sent by Alice before it reached Bob. She knows it is a date and she is very confident that the message is `12.12.2025` as this marks the suspected final date of Eve's mission. She then tries to compute the key. 

Use your knowledge from the lecture about OTP. Think about how Eve can reconstruct a key using the given information. Provide the key Eve reconstructed as the return of the function `return_reconstructed_key() -> str`. **You can either write code that computes it or just return the answer.**

In [7]:
def return_reconstructed_key() -> str:
    return xor_binary_messages(encrypted_date,to_binary("12.12.2025"))

In [8]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(return_reconstructed_key())

00000010101101010110101110010000101001011000000100101100100010000101111011011100


In [9]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

Eve is not sure if the reconstructed key is correct and she was unable to decrypt any other messages from Alice to Bob in a way that would make any sense. However, she wants to make sure that Bob does not get correct dates from Alice anymore. 
As she knows that Alice is just sending dates and these dates are not the first or last day of a month, because all spies worldwide are off on these days, Eve wants you to modify the messages so that the date is moved by one day in any direction.

Implement a function that modifies an encrypted date in the explained way and returns the modified message as return to the function `modify(message:str) -> str`. The argument `message` will be an encrypted date.

In [10]:
def modify(message: str) -> str:
    return xor_binary_messages(message, "00000000000000010000000000000000000000000000000000000000000000000000000000000000")

In [11]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

playground_secret_key = '00000001101100110110101110010000101001011000000100101100100010000101111011011100'

playground_encrypted_date = encrypt("24.12.2025", playground_secret_key) # Sent by Alice, but intercepted by Eve.
print(f"Encrypted date: {playground_encrypted_date}")

playground_modified_ciphertext = modify(playground_encrypted_date) # What Eve sends to Bob.
print(f"Modified ciphertext: {playground_modified_ciphertext}")

playground_decrypted_date = decrypt(playground_modified_ciphertext, playground_secret_key) # Received by Bob. 
print(f"Received by Bob: {playground_decrypted_date}")

Encrypted date: 00110011100001110100010110100001100101111010111100011110101110000110110011101001
Modified ciphertext: 00110011100001100100010110100001100101111010111100011110101110000110110011101001
Received by Bob: 25.12.2025


In [12]:
playground_secret_key = '00000001101100110110101110010000101001011000000100101100100010000101111011011100'
playground_encrypted_date = encrypt("24.12.2025", playground_secret_key) # Sent by Alice, but intercepted by Eve.
playground_modified_ciphertext = modify(playground_encrypted_date) # What Eve sends to Bob.
playground_decrypted_date = decrypt(playground_modified_ciphertext, playground_secret_key) # Received by Bob. 
assert playground_decrypted_date == '25.12.2025' or playground_decrypted_date == '23.12.2025'
# Your code is not necessarily correct just because it passes this test.

del playground_secret_key
del playground_encrypted_date
del playground_decrypted_date
del playground_modified_ciphertext

In [13]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 1.3 - Decrypting the communication (2 Points)

Eve was able to intercept even more messages from Alice. She successfully modified them and forwarded them to Bob. However, Eve is confident that Alice has a limited OTP key only, so she suspects that Alice is reusing it for every new message. Eve also knows that the communicated dates are in December 2025 and one of the intercepted dates is either the 18.12.2025 or the 04.12.2025. 

You have learned of the OTP's weakness when using the same key again in the lecture and now want to impress Eve by decrypting the messages even without knowledge of the key:


Implement a function to reconstruct the messages Alice sent. There are two messages given and both contain a date. The dates are both in December 2025. Under the assumption that for the encryption the same key was used, exploit the XOR construction of OTP to reconstruct the messages.

Implement the function `get_both_dates(date1: str, date2) -> (str, str)` that returns both dates. The arguments and variables `date1` and `date2` are the encrypted dates. The order of them is not important. Use the given function `check_for_date(date: str) -> bool` to check for a valid date automatically.

In [14]:
date1 = '10100111100011100010000101110010010000100001110001101000010010010110111011110000'
date2 = '10100100100000000010000101110010010000100001110001101000010010010110111011110000'

import re

def check_for_date(date: str) -> bool:
    return all(char in '0123456789.' for char in date) and re.match(r"^(0[1-9]|[12][0-9]|3[01])\.(0[1-9]|1[0-2])\.(\d{4})$", date)

In [15]:
def get_both_dates(date1: str, date2: str) -> (str, str):
    xor_plaintexts = xor_binary_messages(date1, date2)
    suspect_dates = ['18.12.2025', '04.12.2025']

    for date in suspect_dates:
        suspect_bin = to_binary(date)
        second_date_bin = xor_binary_messages(xor_plaintexts, suspect_bin)
        second_date = from_binary(second_date_bin)
        
        if check_for_date(second_date):
            return (date, second_date)

In [16]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.
print(get_both_dates(date1, date2))

('18.12.2025', '26.12.2025')


In [17]:
assert get_both_dates(date1, date2) == ('26.12.2025', '18.12.2025') or get_both_dates(date1, date2) == ('18.12.2025', '26.12.2025')
# Your code is not neccessarily correct just because it passes this simple test case.

In [18]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 1.4 - The new spy (1.5 point)

Thanks to your outstanding work, Alice got caught. However, a new spy named Carol emerged soon after, stealing top secret information from your agency and causing even more chaos than Alice! And she has learned from Alice's mistake: Unlike Alice, Carol uses a new uniformly random key for each message.

One day, you come across captured communication between Carol and Bob, who does not encrypt his messages for some reason:

* `19.12.2025: Bob: Carol, when do we meet?`
* `19.12.2025: Carol: 11010010010100001010010101000100111010000101110000011000000111011111001011101100`
* `22.12.2025: Bob: Great meeting! When will we steal the important information again?`
* `22.12.2025: Carol: 11010010010101111010010101000100111010000101110000011000000111011111001011101100`

You notice that the XORing the message `22.12.2025` onto the first ciphertext, and then using the resulting key to "decrypt" the second message yields the message "25.12.2025", as shown by the code below:

In [19]:
print(from_binary(xor_binary_messages(
    xor_binary_messages(to_binary("22.12.2025"), 
                        "11010010010100001010010101000100111010000101110000011000000111011111001011101100"), 
    "11010010010101111010010101000100111010000101110000011000000111011111001011101100")))
# Outputs 25.12.2025

25.12.2025


Have you learned the date Carol and Bob will steal some important information? **Justify** your answer.

No (0.5 points). As the key used for the second message is independent from the key used for the first message, and both keys are uniformly random, the second ciphertext is perfectly hiding and does not reveal anything about the date. (1 point)

Alternative formulations for the justification:
* Carol uses the OTP correctly => It's perfectly secure (1 point)
* The XOR is just a coincidence (1 point)

In [20]:
# You should run this cell when you are finished with task 1. It cleans up a little bit as you should not use the methods from Task 1 in the following tasks.
if "xor_binary_messages" in locals():
    del xor_binary_messages
if "to_binary" in locals():
    del to_binary
if "from_binary" in locals():
    del from_binary
if "encrypt" in locals():
    del encrypt
if "decrypt" in locals():
    del decrypt
if "modify" in locals():
    del modify

## 2. Modes of Encryption (6.5 Points)

While the OTP is perfectly secure, your intelligence agency had significant issues with the key management. It is just too hard to exchange new, perfectly random keys every time. Therefore, they want to move on to modern block ciphers such as AES. 

### Task 2.1 (1.5 Points)

The initial approach used by your agency was to convert the plaintext into a series of blocks, and then encrypt each block individually using the block cipher. You immediately recognize this mode of operation as the *Electronic Codebook Mode*, and know that it is not secure. **Describe** the main problem of the *Electronic Codebook Mode*, what an attacker can learn about an ECB-encrypted message, and how this is compensated using *Cipher Block Chaining Mode* or *Counter Mode*.

Problem:

Identical plaintext blocks generate identical ciphertext blocks when using the same key.

An attacker can detect patterns and repetitions in the plaintext by observing identical ciphertext blocks.


CBC:

Introduces chaining by XORing each plaintext block with the previous ciphertext block (or the initialization vector (IV) for the first block to ensure that if two messages with the same starting block are never same) before encryption.

Identical plaintext blocks produce different ciphertext blocks thanks to feedback from the previous block.


CTR:

Converts the block cipher into a stream cipher and uses a counter value generated value to encrypt.

Since the counter value is unique for each block, a unique keystream is generated, which means that identical plaintext blocks are encrypted differently.

### Task 2.2 (1 Points)

**Explain** why the *Cipher Block Chaining Mode* and the *Counter Mode* need an *Initialization Vector* (IV). Additionally, **describe** a problem that can occur when re-using the IV.

The IV is needed to ensure that the encryption is non-deterministic.
Reusing an IV-key pair breaks security in both modes.

CBC:

The IV is XORed with the first plaintext block to randomize the starting point of the chain formation.

CTR:

The IV (often referred to as a nonce here) is used as the starting value for the counter.

### Task 2.3 (2 Points)

Convinced by your arguments, the agency decides to try out two other modes of operation, the first one being the *Cipher Block Chaining* (CBC) mode.
In CBC, the encrypted output of the cipher from the previous block is XORed into the encryption of the next plaintext block. In the following, you can show that you are an expert in encryption modes by implementing a (simplified) CBC mechanism.

Implement the functions `CBC_encrypt(message, iv, k)` and `CBC_decrypt(ciphertext, iv, k)` which encrypt/decrypt a given text according to an initialization vector `iv` and a key `k` in CBC mode. They should both return a string with the encryption/decryption result. Ensure that your functions only accept texts that are a multiple of 8 characters and throw a `ValueError` otherwise (we disregard anything about padding here).

Use the following encryption/decryption functions to encrypt/decrypt a single block of length `l=8`. Use the new `xor_strings` method if you want to XOR two strings, and do not use the `xor_binary_messages` from the prior exercises anymore.

*Hint:* your `iv` and `k` need to have the same length as a singular encrypted block.

In [1]:
def xor_strings(a: str, b: str) -> str:
    return "".join(chr(ord(x) ^ ord(y)) for x, y in zip(a, b))

# The agency will replace these functions with secure ciphers later.
# The main purpose of this insecurity is to ensure that you get "printable" output characters, makes debugging easier.
def encrypt_block(message: str, key: str) -> str:
    if len(message) != 8:
        raise ValueError("Message length must be 8!")
    if len(key) != 8:
        raise ValueError("Key length must be 8!")
    
    tmp = xor_strings(message, key)
    return tmp[1:] + tmp[0] 

# The agency will replace these functions with secure ciphers later.
# The main purpose of this insecurity is to ensure that you get "printable" output characters, makes debugging easier.
def decrypt_block(ciphertext: str, key: str) -> str:
    if len(ciphertext) != 8:
        raise ValueError("Ciphertext length must be 8!")
    if len(key) != 8:
        raise ValueError("Key length must be 8!")
    
    tmp = ciphertext[-1] + ciphertext[:-1] 
    return xor_strings(tmp, key)

In [22]:
def CBC_encrypt(message: str, iv: str, k: str) -> str:
    if len(message) % 8 != 0:
        raise ValueError("Message length must be a multiple of 8 (no padding is implemented).")

    ciphertext = ""
    previous_block_cipher_text = iv
    
    for i in range(0, len(message), 8):
        current_plaintext_block = message[i:i + 8]
        current_ciphertext_block = encrypt_block(xor_strings(current_plaintext_block, previous_block_cipher_text), k)
        ciphertext += current_ciphertext_block

        previous_block_cipher_text = current_ciphertext_block

    return ciphertext

In [23]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(CBC_encrypt("testtesttesttest", "abcdefgh", "ijklmnop"))

m{||m{l|tddtppxp


In [24]:
# There's a good chance that your code is not correct, even if it passes this test case.
# Take a look at the error handling, and test that yourself :)
assert CBC_encrypt("testtesttesttest", "abcdefgh", "ijklmnop") == "m{||m{l|tddtppxp"

In [25]:
def CBC_decrypt(ciphertext: str, iv: str, k: str):
    if len(ciphertext) % 8 != 0:
        raise ValueError("Ciphertext length must be a multiple of 8.")

    plaintext = ""
    previous_ciphertext_block = iv

    for i in range(0, len(ciphertext), 8):
        current_ciphertext_block = ciphertext[i:i + 8]
        current_plaintext_block = xor_strings(decrypt_block(current_ciphertext_block, k), previous_ciphertext_block)
        plaintext += current_plaintext_block

        previous_ciphertext_block = current_ciphertext_block

    return plaintext

In [26]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(CBC_decrypt("m{||m{l|tddtppxp", "abcdefgh", "ijklmnop"))

testtesttesttest


In [27]:
# There's a good chance that your code is not correct, even if it passes this test case.
# Take a look at the error handling, and test that yourself :)
assert CBC_decrypt("m{||m{l|tddtppxp", "abcdefgh", "ijklmnop") == "testtesttesttest"

In [28]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [29]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 2.4 (2 points)

The other mode of operation the agency wants to try out is a less known one: The *Output Feedback Mode* (OFB). The main idea here is that we can generate a keystream through encrypting the initialization vector again and again. Then, we only need to XOR the plaintext with the keystream. This means, we can even pre-generate such a keystream!

For this task, you can again use the functions `encrypt_block` and `decrypt_block` as given above.

However, this time, you have to implement the functions `OFB_encrypt(message, iv, k)` and `OFB_decrypt(message, iv, k)` which encrypt/decrypt a given text according to an initialization vector `iv` and a key `k` in the simplified Output Feedback (OFB) mode. 

They should both again return a string with the encryption/decryption result. Also, ensure that your functions only accept texts that are a multiple of 8 and throw a `ValueError` otherwise (we disregard anything about padding again).

Encryption and decryption in OFB work as follows:

$C_i = E^i(IV, k) \oplus P_i$

$P_i = C_i \oplus E^i(IV, k)$

where $E^i(IV, k)$ means that the $IV$ is encrypted $i$ times with the key $k$, as also shown in the image below. 

![](./ofb.png)

In [30]:
def OFB_encrypt(message: str, iv: str, k: str) -> str:
    if len(message) % 8 != 0:
        raise ValueError("Message length must be a multiple of 8 (no padding implemented).")

    output = ""
    keystream_block_input = iv
    
    for i in range(0, len(message), 8):
        current_keystream_block = encrypt_block(keystream_block_input, k)
        current_plaintext_block = message[i:i + 8]
        current_ciphertext_block = xor_strings(current_plaintext_block, current_keystream_block)
        output += current_ciphertext_block

        keystream_block_input = current_keystream_block

    return output

In [31]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(OFB_encrypt("abcdefghijklmnop", "AAAAAAAA", "abcdefgh"))

B@F@B@NH),+.- '2


In [32]:
# There's a good chance that your code is not correct, even if it passes this test case.
# Take a look at the error handling, and test that yourself :)
assert OFB_encrypt("abcdefghijklmnop", "AAAAAAAA", "abcdefgh") == "B@F@B@NH),+.- '2"

In [33]:
def OFB_decrypt(ciphertext: str, iv: str, k: str) -> str:
    if len(ciphertext) % 8 != 0:
        raise ValueError("Ciphertext length must be a multiple of 8.")

    output = ""
    keystream_block_input = iv
   
    for i in range(0, len(ciphertext), 8):
        current_keystream_block = encrypt_block(keystream_block_input, k)
        current_ciphertext_block = ciphertext[i:i + 8]
        current_plaintext_block = xor_strings(current_ciphertext_block, current_keystream_block)
        output += current_plaintext_block
        
        keystream_block_input = current_keystream_block

    return output

In [34]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.

print(OFB_decrypt("B@F@B@NH),+.- '2", "AAAAAAAA", "abcdefgh"))

abcdefghijklmnop


In [35]:
# There's a good chance that your code is not correct, even if it passes this test case.
# Take a look at the error handling, and test that yourself :)
assert OFB_decrypt("B@F@B@NH),+.- '2", "AAAAAAAA", "abcdefgh") == "abcdefghijklmnop"

In [36]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [37]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

In [38]:
# Run this cell when you are finished with task 2. It cleans up the functions which you will not need in the following task.
if "xor_strings" in locals():
    del xor_strings
if "CBC_encrypt" in locals():
    del CBC_encrypt
if "CBC_decrypt" in locals():
    del CBC_decrypt
if "OFB_encrypt" in locals():
    del OFB_encrypt
if "OFB_decrypt" in locals():
    del OFB_decrypt

## 3. The Galactic Length Extension Attack (7 Points)

In the distant future, aboard the spaceship Crypto Voyager, Captain Cipher and his trusty sidekick, Data Droid, are on a mission to protect their valuable cargo of intergalactic supplies from space pirates. They have a strict protocol for sending messages about their cargo status to ensure everything is secure. This includes a protocol to open their hangar used to store cargo on week-ends.

For the protocol to be flexible, the protocol works on a "key-value" basis, i.e., each message consists of a list of key-value pairs. For example, `time=251007-1005;hangar=open;` opens the hangar, but only if the time is correct. Malformed key-value pairs are assumed to be padding and thus are ignored.

They use the following function for parsing the messages:

In [39]:
def parse_key_value_pairs(message: str) -> dict[str, str]:
    result = {}
    for encoded_pair in message.split(";"):
        parts = encoded_pair.split("=")
        if len(parts) == 2:
            result[parts[0]] = parts[1]
    return result

### Task 3.1 Creating MACS (2 points)

One day, Captain Cipher receives an alert that there are pirates nearby. "We must secure our messages with a MAC!" he exclaims. "If anyone tries to tamper with our hangar, we’ll be in big trouble!". Data Droid agrees but needs help implementing the CBC MAC.

Your task is to assist Data Droid in implementing the function `create_MAC`, which accepts a message as `bytes` (a read-only byte-array), a key as `bytes` and computes a CBC-MAC **without the final masking** using AES-128. The MAC should be returned as `bytes`. Use the library [Cryptography](https://cryptography.io/en/latest/) that provides many cryptographic algorithms. 

Hint: CBC-MAC works by setting the `IV` to all zeros.

In [ ]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.ciphers.algorithms import AES128
from cryptography.hazmat.primitives.ciphers.modes import CBC

def create_MAC(message: bytes, key: bytes) -> bytes:
    block_size_bytes = 16
    iv = b'\x00' * block_size_bytes
    
    #padding
    padded_data = message

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    mac = ciphertext[-block_size_bytes:]
    return mac

In [49]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.
print(create_MAC(b"time=251007-1005;hangar=open;pad", b"supersecretkeyyy"))

b'&\xde.\x04\x08V\x194\xf3\x8c\xb7\x7fs9!R'


In [50]:
assert create_MAC(b"time=251007-1005;hangar=open;pad", b"supersecretkeyyy") == b'&\xde.\x04\x08V\x194\xf3\x8c\xb7\x7fs9!R'

In [51]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 3.2 Checking the MAC (1 point)

After securing their message format (plaintext message, MAC), Captain Cipher insists that they need a verification function as well. "These MACs are useless if they are not checked by our hangar!"

Can you help them implement this verification function? It should check if the received message matches what was originally sent by verifying the MAC.

In [52]:
def check_MAC(message: bytes, mac: bytes, k: bytes) -> bool:
    return mac==create_MAC(message, k)

In [53]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.
playground_key = b"supersecretkeyyy"
message_1 = b"time=251007-1005;hangar=open;pad"
message_2 = b"time=251007-2055;hangar=open;pad"
mac = create_MAC(message_1, playground_key)

print(check_MAC(message_1, mac, playground_key))
print(check_MAC(message_2, mac, playground_key))

True
False


In [54]:
playground_key = b"supersecretkeyyy"
message_1 = b"time=251007-1005;hangar=open;pad"
message_2 = b"time=251007-2055;hangar=open;pad"
mac = create_MAC(message_1, playground_key)

assert check_MAC(message_1, mac, playground_key) is True
assert check_MAC(message_2, mac, playground_key) is False

# These are just "playground variables", and should not be used in the following cells!
del playground_key
del message_1
del message_2
del mac

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

### Task 3.3 - Opening the hangar (4 points)

Unbeknownst to Captain Cipher and Data Droid, Pirate Pete has been eavesdropping on their communications.
He suspects that it is possible to forge some message-tag pairs given a set of already valid message-tag pairs, and is now trying to unlock the hangar based on the following two messages:

- `time=251007-2045;hangar=open;pad`
- `time=251007-2115;hangar=closed;p`

If he is fast enough, he may be able to combine both messages and their MACs such that the hanger will open for him, allowing him to steal the cargo.
Luckily for him, the parsing method used by Captain Cipher and Data Droid is very generous towards malformed inputs, such that messages like `time=251007-2115;hangar=closed;p<gibberish>;hangar=open;pad` will still open the hangar.

Your task is to write a function `construct_forge` that accepts the opening and closing messages (as `bytes`) and their corresponding MACs (as `bytes`) and returns a (message, MAC)-tuple of a message that will open the hangar. You can use the `xor_bytes` function for the bitwise XOR of two `bytes`.

In [55]:
def xor_bytes(a: bytes, b: bytes) -> bytes:
    return bytes([x^y for x, y in zip(a, b)])

def construct_forge(open_message: bytes, open_mac: bytes, close_message: bytes, close_mac: bytes) -> tuple[bytes, bytes]:
    manipulated_second_block = xor_bytes(close_message, open_mac)
    forged_message = open_message + manipulated_second_block
    forged_mac = close_mac
    return (forged_message, forged_mac)

In [57]:
### PLAYGROUND
# You can use this cell to test out your implementation. Everything in this cell will be ignored during grading.
import random

class Hangar:
    __key: bytes # Not known by the pirates.

    def __init__(self):
        # Even when implemented correctly, the attack can run into issues due to <gibberish> not being UTF-8 decodable.
        # We have pre-computed some keys where the attack works, but the grading test will use truely random (working) keys, 
        # so brute-forcing the key based on this key list will fail the grading test!
        self.__key = random.choice([b'Z$c\xbao18p\x0b\xfco$\x0f\xf9\x0bb', b'7!\x81\xcaZ.\x86\xa8\xb7j\x9bM5\x87Af', 
                                    b'\xa2\xdb\xf3C;b\xfb\xe7`\x8eg\x06F<\x02\x84', b'c\xd4X\x18\xa5\x12\x8dMo\x8e2\x9b\xbeu\x03\xb0', 
                                    b'\xb2\xb5\xe94\xf9\x06\xbe\xb2]Z\xe6\xd9ah\x91\xa9', b'1\x1b\xd2\xad|\xd1\xbd8v\x8b:ay\xc3\xc3\x8a', 
                                    b'\x17\x9f\xa43\xc7\x8c\xb7@\xc8\xce\x1fU\t\x87\x03\xec', b'\xba\x00`\\);\x85\xd7v\x7f\x7fpo\xd4\xf7~', 
                                    b'K\r\xea\x90\xddv\x8bv\x84\x0f+\xe9\xce#\x94V', b'5\xd2F.KP\x1b\xe0\xf7\x8a\x82\x17>K\x99r', 
                                    b'\xe9U\x01\xeb\xd8,\x12m\xfe\xa2\x8c0\x89\xf0\xf1\x8e', b'\xd0q\xcc\x13\x90\xe4x\x05\xcd\xf3\x05\xf8\xaf\xb6\x92\xbb', 
                                    b'2\x16\x08B\xbe\xaaY\xbd\xea\xac\x85\x9e\xa4\xca\x8f\xe0', b'\xf0\xf2\x8f\x96\xda\xae\xdf00L\xf3\xf5\xa5S\xae ', 
                                    b'\xd6\xc0@\x93\x88M\xa8\xd91\xa4\xdc\x07\x15\xf1\x12p', b'\x0c[^\xfb\xc0\x95\x1a\xed\xf7`\xe5%\xdb\xb6\x173', 
                                    b'\x97\xbf\x06\x8b.\xd5\x91\xe7BG\xa5\xda]\x88\xf9\x86', b'\x8a\t\x0e,\xbbJ\xb9\xb5\x003\xc2\x94\xbe\xec\xa5\x9a', 
                                    b'\x98\x96\xc2\x80\xbd\xbb\x05\x14\xfe\x1axe2x50', b'\xe2\xec=R\x91C\xc2t:\xf6\xc8\xf4\xce\xc0\xef!', 
                                    b'\xd9\xac\xff\x03F\x15\x9e\xe2/\xe6\xe7\xa0K\xf3\xd3\xd9', b'_\x96NF\t\xab\xe3\x05wJ\xa3\x18?mEW', 
                                    b'\xfe\xb7\xcfl\xab3\x0bVX\x9d\xb8\xcb\xd4\x85b\xc7', b'\x08\xc2,\x0e+\xc6+\xa6\x9c/\x86\xbe*\x1c1@', 
                                    b'%\xe3\xcb\xb7-A*\x9c1\xe4\xd5\x89\x80$\x92\xd0', b'\x81\x1b\xaf\xcb=\x19\xb6\x92hm\xf0\x08\x0b\x7f\x1bS', 
                                    b'\x89\x9f\xf9\xd5\xb7\xb1\xff\xd5[\xa7\x8c\x94\xbbL\xf3\xe0', b';\xdfd\x03\xe7\x0f\x17x\xa0\x14k\xa1*\xb6F\x8f', 
                                    b'ZG\xb3\xa6\x12\xf51\x1e\xe3\xe6{\x8d\x8d\xf0\xf9\x10', b'>\xf2\xbe:\xbfW\xa6\xbc\xaf\xc7\xa5\x97\xc5\xbf\xf4\x0c', 
                                    b'AiX\xd6\xb1Y K\x07\xc4\xab\xfe\xa3|\xd2.', b'\xb6\xb1\x1cw\xcd|8]b\xf1]\x95@\xba\xe0g', 
                                    b'\xb2_s\xf9\xc0\x03\xb6S2\xf3\x05\xad\x95c\x7f\xd6', b'\xe3\xc7>\xda\xab\xe3\xb5\xed\x8a.\xfd\xd9\x08\x94#M', 
                                    b'\xc9\x9b>\x87\xef\x82A"\xc4\xa0+V\xc2g|\xea', b'\xe8@\xc8&\xc5\xd7De\x80R\xb0\xa8\xfb\xd4O\xde', 
                                    b'$W\xe9\x99\x14\xe8%\x06\x83\x04j@$\x9d\xba\x12', b'\x82\x86Tk\x1f\x04:\x04\xa7w\x1e\x81\xce\xc3=\x02', 
                                    b'\xb8\x03\xeb>,!\x8dr\x93\xcc\x9b-Z\xea\xae^', b'\x9c\xf4HVMD|\xb0\xea\xc2d\xa3C`CF', 
                                    b'\xde\x95\xc9]\xe1\xf9\x05\xbe\xa2f\xd3P\xdf\xda\xc7\x0c', b'`\xb2\xd4\xa0\x1aB\x81\x98\xe1?\xacV\xd4\x87\xb7\x11', 
                                    b'\x1aHz\xb7)\xaeX\xaf\xa9\xf2\x05z\xd8\xc8Tj', b'\x90\xf3nQ\x18\xdd\xf0\x16\x13da\xd9\x11Gk/', 
                                    b'\x0e\xfb\xb8z\x10\xbf?!\xbe\xe7\xc3\xac\xd1\x16xL', b'\x87\x81\xd0b`s\x17\xc5\xec\x16M[\x06]H]', 
                                    b'\xecg\x01\xdf\xbdxkW\x89f=\xc0h}Cy', b'\xd6\xb0\x17#i\xbe\xdf\x0f\n\xebD\xe6\xfd\x10\xba\xec', 
                                    b'o\xc8\x9e\xc1g\xb1\xee\xd0d\xa8v\xed\x14n\x82\x9d', b'Y\x8b\xe1e(~)N\xfd\x05\xa6\xc2\xc2f\xcd3', 
                                    b"\xbb7\xf1\xe9\xb8t5AWy^\x99'\x0e\x0b\x84", b'\xa3\xf4\x02\x10\xab\xe7W)DuA\xf8\x8d\xd7\x1e\xa0', 
                                    b'g\x0c\xb7\x9b>I\x864}0\xc6t,_\xa8\xfb', b'6\x16\xe1\xcbL\x1a|\xf2\x18\xe9"c\xea~\x98\xb3', 
                                    b'lq\xd7\x92\xabw\xda\xf7\xa2\xe2\x07\xef\xbe5Lp', b'\x0f\xbcd\x9d\x7f\x81\xfd1\\\xb7xe\xe3\x85Ek', 
                                    b"*\x0e\xc1\xc98\xbf4q\x1f\xc1\x99(\xbe'\xc7I", b'\xe8\x11\x810t*\x8eX@\xb5u0\x8c\xd2\x01\x89', 
                                    b'\x1a\x88\x980\xaa\x0e\x1dd\x13\x9fwixDx\xe8', b'\xe0`oV\x9eF`\xf3c\x9e&\xe1\xda\x19\xcbi', 
                                    b'\x1b\xddod\x82\xaa\xf8\x1e\xf3\xd4\x92\xb1\xe7p\tJ', b'\x97\x9e\xa6M\xd6\x8d^\xb1\xe8\x8a\x03hv)h\x9a', 
                                    b'\xad\xed\xef\xfe\xaf\xeb\xa3\xdeW\xf1g4}lW\xbb', b'\x80-\xf1p\xce=\xf0ml\xe6\xf4\x1a\xde\x00\xa5\xc8'])

    def generate_eavesdropped_open_message(self) -> tuple[bytes, bytes]:
        return b"time=251007-2045;hangar=open;pad", create_MAC(b"time=251007-2045;hangar=open;pad", self.__key)
    
    def generate_eavesdropped_close_message(self) -> tuple[bytes, bytes]:
        return b"time=251007-2115;hangar=closed;p", create_MAC(b"time=251007-2115;hangar=closed;p", self.__key)

    def handle_message(self, message: bytes, mac: bytes) -> bool:
        if not check_MAC(message, mac, self.__key):
            print("[!] MAC check failed!")
            return False
        parsed = parse_key_value_pairs(message.decode('utf-8'))
        if parsed['time'] != '251007-2115':
            print("[!] Replay of old message detected!")
            return False
        
        if parsed['hangar'] == "open":
            print("[+] Opening the hangar")
            return True
        elif parsed["hangar"] == "closed":
            print("[+] Closing the hangar.")
            return False
        else:
            print("[!] Unknown hangar command!")
            return False
        
hangar = Hangar()
open_message, open_mac = hangar.generate_eavesdropped_open_message()
close_message, close_mac = hangar.generate_eavesdropped_close_message()

forged_message, forged_mac = construct_forge(open_message, open_mac, close_message, close_mac)
print(forged_message, forged_mac)

assert hangar.handle_message(forged_message, forged_mac)

b'time=251007-2045;hangar=open;pad~\xdeB\x16\xd2\xc0e\xac\x90\xff\xcd\xd6\xbf(Nf' b'P5T9L&\xf0\xae3J\x12\x010\xf4\xa98'
[!] MAC check failed!


AssertionError: 

In [ ]:
# Even this cell seems empty, it contains automatic tests. Please do not remove this cell and just ignore it.

## 4. Make sure that your code is working (0 points)

Please make sure that your code is working, i.e., check whether
- you import only built-in Python libraries and libraries provided by us,
- your code is compatible with Python 3.12,
- restarting the Jupyter kernel and re-running all code cells works without errors,
- all assertions provided by us pass.

Technically, this is not a task, just a gentle reminder to make sure that your previous solutions are working :)

## 5. Feedback (0.5 points)

You made it through another assignment! Since we want to know how it went and how we might improve the exercises, we include the following task. Here, you can write constructive feedback! You even get 0.5 points for it if you write anything. But don't worry, we do not grade the content itself!

Very nice structure but the last exercise was kinda confusing.

Nevertheless it was sometimes confusing when someone forgot to run every single prewritten codeline.

9/10 homework experience.